In [1]:
import glob
import os
import struct
from nltk.tokenize import sent_tokenize

from tensorflow.core.example import example_pb2

In [2]:
file0 = '/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/train_000.bin'

In [18]:
files = sorted(os.listdir('/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/'))

In [19]:
files

['test_000.bin',
 'test_001.bin',
 'test_002.bin',
 'test_003.bin',
 'test_004.bin',
 'test_005.bin',
 'test_006.bin',
 'test_007.bin',
 'test_008.bin',
 'test_009.bin',
 'test_010.bin',
 'test_011.bin',
 'train_000.bin',
 'train_001.bin',
 'train_002.bin',
 'train_003.bin',
 'train_004.bin',
 'train_005.bin',
 'train_006.bin',
 'train_007.bin',
 'train_008.bin',
 'train_009.bin',
 'train_010.bin',
 'train_011.bin',
 'train_012.bin',
 'train_013.bin',
 'train_014.bin',
 'train_015.bin',
 'train_016.bin',
 'train_017.bin',
 'train_018.bin',
 'train_019.bin',
 'train_020.bin',
 'train_021.bin',
 'train_022.bin',
 'train_023.bin',
 'train_024.bin',
 'train_025.bin',
 'train_026.bin',
 'train_027.bin',
 'train_028.bin',
 'train_029.bin',
 'train_030.bin',
 'train_031.bin',
 'train_032.bin',
 'train_033.bin',
 'train_034.bin',
 'train_035.bin',
 'train_036.bin',
 'train_037.bin',
 'train_038.bin',
 'train_039.bin',
 'train_040.bin',
 'train_041.bin',
 'train_042.bin',
 'train_043.bin',
 'tr

In [72]:
reader = open(file0, 'rb')
len_bytes = reader.read(8)
str_len = struct.unpack('q', len_bytes)[0]

In [73]:
example_string = struct.unpack("%ds" % str_len, reader.read(str_len))

example_str = example_string[0]
content = example_pb2.Example.FromString(example_str)

In [74]:
content

features {
  feature {
    key: "abstract"
    value {
      bytes_list {
        value: "<s> mentally ill inmates in miami are housed on the `` forgotten floor \'\' </s> <s> judge steven leifman says most are there as a result of `` avoidable felonies \'\' </s> <s> while cnn tours facility , patient shouts : `` i am the son of the president \'\' </s> <s> leifman says the system is unjust and he \'s fighting for change . </s>"
      }
    }
  }
  feature {
    key: "article"
    value {
      bytes_list {
        value: "editor \'s note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o\'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , \'\' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbe

In [13]:
example_str

b"\n\xd6#\n\xdf\x02\n\x08abstract\x12\xd2\x02\n\xcf\x02\n\xcc\x02<s> mentally ill inmates in miami are housed on the `` forgotten floor '' </s> <s> judge steven leifman says most are there as a result of `` avoidable felonies '' </s> <s> while cnn tours facility , patient shouts : `` i am the son of the president '' </s> <s> leifman says the system is unjust and he 's fighting for change . </s>\n\xf1 \n\x07article\x12\xe5 \n\xe2 \n\xdf editor 's note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbed the `` forgotten floor . '' here , inmates with the most severe mental illnesses are inca

In [12]:
example_string

(b"\n\xd6#\n\xdf\x02\n\x08abstract\x12\xd2\x02\n\xcf\x02\n\xcc\x02<s> mentally ill inmates in miami are housed on the `` forgotten floor '' </s> <s> judge steven leifman says most are there as a result of `` avoidable felonies '' </s> <s> while cnn tours facility , patient shouts : `` i am the son of the president '' </s> <s> leifman says the system is unjust and he 's fighting for change . </s>\n\xf1 \n\x07article\x12\xe5 \n\xe2 \n\xdf editor 's note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbed the `` forgotten floor . '' here , inmates with the most severe mental illnesses are inc

In [11]:
content

features {
  feature {
    key: "abstract"
    value {
      bytes_list {
        value: "<s> mentally ill inmates in miami are housed on the `` forgotten floor \'\' </s> <s> judge steven leifman says most are there as a result of `` avoidable felonies \'\' </s> <s> while cnn tours facility , patient shouts : `` i am the son of the president \'\' </s> <s> leifman says the system is unjust and he \'s fighting for change . </s>"
      }
    }
  }
  feature {
    key: "article"
    value {
      bytes_list {
        value: "editor \'s note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o\'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , \'\' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbe

In [ ]:
article  = str(content.features.feature['article'].bytes_list.value[0])
abstract = str(content.features.feature['abstract'].bytes_list.value[0])

abstract_clean, article_clean = sent_tokenize(abstract), sent_tokenize(article)

In [ ]:
def getsentences(file0):
    reader = open(file0, 'rb')
    len_bytes = reader.read(8)
    str_len = struct.unpack('q', len_bytes)[0]
    example_str = struct.unpack("%ds" % str_len, reader.read(str_len))[0]
    content = example_pb2.Example.FromString(example_str)
    

    article  = str(content.features.feature['article'].bytes_list.value[0])
    abstract = str(content.features.feature['abstract'].bytes_list.value[0])

    return sent_tokenize(abstract), sent_tokenize(article)